In [2]:
import os
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 定义图像文件夹路径和类别标签
data_dir = 'isic'
categories = ['f1', 'f2']

# 将图像文件转换为特征向量
def extract_features(img):
    # 使用OpenCV将图像转换为灰度图，并resize到指定大小
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (50, 50))
    # 将灰度图拉成一维向量
    features = np.reshape(gray, (1, -1))
    return features

# 加载数据集
data = []
labels = []
for category in categories:
    category_dir = os.path.join(data_dir, category)
    for filename in os.listdir(category_dir):
        img_path = os.path.join(category_dir, filename)
        img = cv2.imread(img_path)
        if img is not None:
            features = extract_features(img)
            data.append(features)
            labels.append(category)

# 将图像特征和标签列表转换为numpy数组
data = np.concatenate(data, axis=0)
labels = np.array(labels)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# 使用SVM进行分类
clf = svm.SVC(C=1.0, kernel='linear', decision_function_shape='ovr')
clf.fit(X_train, y_train)

# 对测试集进行预测，并计算准确率
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))



Accuracy: 100.00%


In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import rotate, flip

# 假设你已经有了训练数据X_train和对应的标签y_train
# X_train的形状为 (样本数, 特征数)，y_train的形状为 (样本数,)

# 数据增强函数
def augment_data(images, labels):
    augmented_images = []
    augmented_labels = []
    for image, label in zip(images, labels):
        augmented_images.append(image)
        augmented_labels.append(label)

        # 随机旋转图像
        rotated_image = rotate(image, angle=np.random.randint(-30, 30))
        augmented_images.append(rotated_image)
        augmented_labels.append(label)

        # 随机水平翻转图像
        flipped_image = flip(image, axis=1)
        augmented_images.append(flipped_image)
        augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

# 进行数据增强
X_train_augmented, y_train_augmented = augment_data(X_train, y_train)

# 特征归一化
scaler = MinMaxScaler()
X_train_augmented = scaler.fit_transform(X_train_augmented)

# 将数据分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_train_augmented, y_train_augmented, test_size=0.2, random_state=42)

# 创建SVM分类器
svm_classifier = svm.SVC()

# 训练SVM模型
svm_classifier.fit(X_train, y_train)

# 在验证集上进行预测
y_pred = svm_classifier.predict(X_val)

# 计算准确率
accuracy = accuracy_score(y_val, y_pred)
print("验证集准确率：", accuracy)
